In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess registered successfully.
GetSbml registered successfully.
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
import numpy as np 

y = np.array([0.5, 1.0, 0.75, 0.60, 0.66])

average_y = np.mean(y)

# Calculate the MSE for each value in y with respect to the average
mse_scores = (y - average_y) ** 2

mse_scores, average_y

(array([0.040804, 0.088804, 0.002304, 0.010404, 0.001764]), 0.7020000000000001)

In [3]:
"""TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:

{('emitter',): [{
    'copasi': [
        {
            'floating_species_concentrations': {
                'plasminogen': 0.0,
                'plasmin': 0.0,
                'single intact chain urokinase-type plasminogen activator': 0.0,
                'two-chain urokinase-type plasminogen activator': 0.0,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0
        },
        ...
    ],
    'amici': [
        {
            'floating_species_concentrations': {
                'plasminogen': 1.1758171177387002e+16,
                'plasmin': 1096150505274.1506,
                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
                'two-chain urokinase-type plasminogen activator': 80249.33829510311,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0},
        },
       ...
    ]


"""

"TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:\n\n{('emitter',): [{\n    'copasi': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 0.0,\n                'plasmin': 0.0,\n                'single intact chain urokinase-type plasminogen activator': 0.0,\n                'two-chain urokinase-type plasminogen activator': 0.0,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0\n        },\n        ...\n    ],\n    'amici': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 1.1758171177387002e+16,\n                'plasmin': 1096150505274.1506,\n                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,\n                'two-chain urokinase-type plasminogen activator': 80249.33829510311,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0},\n        },\n 

In [4]:
# Step 1: Define the input parameters of the api method

biomodel_id = 'BIOMD0000000630'
sbml_dirpath = '../biosimulator_processes/model_files/sbml'
model_fp = os.path.join(sbml_dirpath, f'{biomodel_id}_url.xml')
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30
n_steps = 42
simulators = ['copasi', 'amici']

In [5]:
biomodel_id = 'BIOMD0000000744'
document = {
    'sbml': {
        '_type': 'step',
        'address': 'local:get_sbml',
        'config': {
            'biomodel_id': biomodel_id
        },
        'inputs': {
                'biomodel_id': ['biomodel_id_store']
        },
        'outputs': {
                'sbml_model_fp': ['sbml_model_fp_store'],                             
        }
    },
    'ode_compare': {
        '_type': 'step',
        'address': 'local:compare_ode_step',
        'inputs': {
            'data': ['sbml_model_fp_store']
        },
        'outputs': {
            'comparison': ['comparison_store']
        }
    },
    'simulation': {
        'copasi': {
            '_type': 'process',
            'address': 'local:copasi',
            'config': {
                'model': {
                    'model_source': 'comparison_store'
                },
                'method': 'lsoda',
                'species_context': 'concentrations'
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'time': ['time_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'time': ['time_store']
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'comparison': 'string',
                    'names': 'list[string]'
                },
            },
            'inputs': {
                'comparison': 'comparison_store',
                'names': ['names_store']
            }
        }
    } 
}
    


In [6]:
def generate_composite(doc: dict) -> Composite:
    return Composite(config={'state': doc}, core=CORE)

def get_output(doc: dict, dur: int = 1) -> dict:
    comp = generate_composite(doc)
    comp.run(dur)
    return comp.gather_results()
    

def run_workflow(biomodel_id: str, dur: int):
    document_a = {
        'sbml': {
            '_type': 'step',
            'address': 'local:get_sbml',
            'config': {
                'biomodel_id': biomodel_id
            },
            'inputs': {
                    'biomodel_id': ['biomodel_id_store']
            },
            'outputs': {
                    'sbml_model_fp': ['sbml_model_fp_store'],                             
            }
        },
        'ode_compare': {
            '_type': 'step',
            'address': 'local:compare_ode_step',
            'inputs': {
                'data': ['sbml_model_fp_store']
            },
            'outputs': {
                'comparison': ['comparison_store']
            }
        },
        'emitter_a': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'comparison': 'string',
                },
            },
            'inputs': {
                'comparison': ['comparison_store'],
            }
        }
        
    }
    print(get_output(document_a, 1))
    """sbml_fp = get_output(document_a, 1)[('emitter_a',)][0]['comparison']
    print(sbml_fp)
    
    document_b = {'copasi': {
            '_type': 'process',
            'address': 'local:copasi',
            'config': {
                'model': {
                    'model_source': sbml_fp
                },
                'method': 'lsoda',
                'species_context': 'concentrations'
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'time': ['time_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'time': ['time_store']
            }
        },
        'emitter_b': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'comparison': 'string',
                    'floating_species_concentrations': 'tree[float]'
                },
            },
            'inputs': {
                'comparison': 'comparison_store',
                'floating_species_concentrations': ['floating_species_concentrations_store'],
            }
        }
    }"""
    # return get_output(document_b, dur)

In [7]:
results = run_workflow(biomodel_id, duration)

results

/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/BIOMD0000000744_url.xml
Error reading SBML file


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
wf = Composite(config={'state': document}, core=CORE)

In [ ]:
wf.run(10)
state = wf.gather_results()

In [ ]:
state

In [ ]:
state

In [8]:
from biosimulator_processes.io import fetch_sbml_file
sbml_file = fetch_sbml_file(biomodel_id)

In [9]:
sbml_file

'/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/BIOMD0000000744.xml'

In [ ]:
from basico import load_model, get_species

model = load_model(sbml_file)


model

In [ ]:
get_species(model=model)